In [ ]:
!pip install rouge-score

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util

# ===========================================================
# ✳️ Change these lists each time for a new model–prompt pair
# ===========================================================
candidates = [ "Here paste the 80 generated NL descriptions "
]

references = [
    "go to the red room , then go to landmark three.",
    "go through the third floor and navigate to the green room.",
    "go to the third floor avoiding the purple room. if you reached the third floor you can stay there, or head to the purple room.",
    "don't go into the green room until going to landmark two.",
    "always avoid the green room and navigate to the third floor.",
    "avoid the red room and go to landmark one.",
    "go to landmark three but remain on the third floor.",
    "go to landmark one by only navigating within the green room.",
    "go to the green room.",
    "navigate directly to landmark one.",
    "enter the blue room through the green room",
    "go through the green room into the blue room",
    "go through the yellow or red room to the green room",
    "go to blue room without going to green room",
    "go to red room but never go to yellow room",
    "go to the green room but avoid the blue room",
    "go to the red room",
    "enter the blue room",
    "go through the yellow or red room to reach the blue room",
    "go through the red or blue room to the green room.",
    "Find Citizen's Bank and then find Chipotle.",
    "Go to CVS and then go to Watson Center.",
    "Find Barus Building and then find CVS.",
    "Do not enter Angell St and find CVS.",
    "Stay away from Angell St and find Barus Building.",
    "Avoid Brook St and first go to Bookstore and then Science Library.",
    "Always visit Nelson Fitness Center, and find Chipotle after that.",
    "Stay in Science Library and then go to Starbucks.",
    "Remain in Barus Building and then visit Watson Center.",
    "Do not enter Brook St but find Science Library.",
    "Avoid Angell St and then find FedEx Office.",
    "Go to Bookstore.",
    "Find where is Chipotle.",
    "Always stay in Science Library and eventually go to CVS.",
    "Remain at Watson Center but eventually go to Starbucks.",
    "after both iwBoSK and NpPUaHQZqPQa, absolutely never IM_CcZj",
    "under no condition PXjEPOci.",
    "eventually, ckojym_vNUR or mOqug and, at some point later, mOqug again",
    "it will happen that qeBtdFQHmb.",
    "aWpSUYgiTCy involves that whenever CAcfkLQXAL occurs, dvWRlxPo eventually happens",
    "as either sYCGfBAVVp or QkktvCMxcVGI, finally, QXjvGmzx will happen",
    "it is always true that vUnUHzC",
    "whenever qYWhgQe then every time both mIEYuyhBx and vWAYect hold.",
    "every time GgvGADnCuwzUs then in the future either IgbAcsPKo or cenfclGT",
    "whenever paPQJQPqQOPwgn then sooner or later GXjrgYRkt_Gn_ or FEsX.",
    "it will not happen that sTqTp or zEmMXnN or ideleiA_fgg.",
    "eternally, either tjVXmfFhIx, AQogUdFHEZkp or mdPPpTdDtZrebD.",
    "finally, either AjZXYF_zM, KfDeQSGCPTEw or BPZqUyExRRq.",
    "if oGeoKTIhZKz_W then it will happen that both lOGrS and BgSIpcmj.",
    "always both dbQVDo and uotjbfVtn implies that it will not happen that syBUlnQ.",
    "first, in the future JLOlbbwc, and then, in the future TkfnNgDBvtbM.",
    "zFVJLgTpxjAEHN involves that if Z_AuRh then it will happen that nKIlX.",
    "eventually, cfZvISle",
    "every time either ERIzBflUWqY, ETbuydMOmBLV or JVygMUXkXT.",
    "it is never the case that ApZGIwL or dXwhbCPbJVMuwUb or vAvKToY.",
    "it is never the case that the escalator moves and together the house is open and the engine starts.",
    "all the time a car enters and together a train derails and the car stops.",
    "it is going to happen that a table has been moved and, at the same time, the train has been launched and a motorbike falls down.",
    "whenever both the car starts and the engine starts then at a certain moment the constructor instantiate objects and the engine stops.",
    "whenever either the brake is released or a table is brown then never, a constructor instantiate objects and a house is open.",
    "absolutely never, either the table is brown, a motorbike has started or a motorbike has stopped.",
    "always, the escalator speeds up and, at the same time, the brake is pressed and the engine starts.",
    "it is going to happen that the motorbike falls down and, at the same time, a motorbike catches fire and the engine starts.",
    "whenever both the bar is down and the brake is released then at some point a train derails and the brake is released.",
    "if the sensor captures data or the brake is released then in any case the brake is released or the motorbike has started.",
    "first, at some point in time the brake is pressed or a motorbike has started, and then, it will happen that a table is brown or the motorbike has started.",
    "when a semaphore is green then always when the sensor retrieves data then at some point the house is built afterwards.",
    "whenever the semaphore is broken and the train is crossing then it will happen that the bridge opens and a train has been launched.",
    "in any case either the sensor gathers information or a house is built.",
    "never, the car enters or the car stops or the engine starts.",
    "it should not happen that a cpu cools down and together the monitor lights up and a model is underfitted.",
    "it is always the case that the model overfits and, at the same time, a computer stops and the monitor lights up.",
    "finally, the monitor lights up and together a model overfits and the fridge is switched on.",
    "after a model underfits, sooner or later either the plane lands or the computer starts.",
    "finally, a computer starts and a model is underfitted and, at a certain moment both a model is underfitted and a cpu is cooled down afterwards.",
    "when the cpu overheats then whenever a mouse captures input then at some point in time the cpu is overheated afterwards.",
    "if the computer stops or a model underfits then eventually, both a cpu overheats and the mouse captures input.",
    "at any time a computer stops and together the monitor shows output and a fridge is switched on.",
    "always either a computer stops or the mouse clicks implies that it will not happen that a plane lands and the fridge is switched on.",
    "under no circumstances a computer starts or the fridge is empty or the monitor shows output."
]

# ===========================================================
# Create the model–prompt pair container (edit each time)
# ===========================================================
model_prompt_pairs = {
    "MXPX": {"candidates": candidates, "references": references}
}

# ===========================================================
# Load SBERT model (only once)
# ===========================================================
print("🔹 Loading SentenceTransformer model...")
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")

# ===========================================================
# Initialize metric utilities
# ===========================================================
smooth = SmoothingFunction()
weights_bleu3 = (1/3, 1/3, 1/3, 0)
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# ===========================================================
# Main Loop
# ===========================================================
for tag, data in model_prompt_pairs.items():
    print(f"\n🚀 Processing {tag}...")

    candidates = data["candidates"]
    references = data["references"]

    # --- BLEU-3 ---
    bleu_scores = []
    for ref, cand in zip(references, candidates):
        ref_tokens = [ref.lower().split()]
        cand_tokens = cand.lower().split()
        score = sentence_bleu(ref_tokens, cand_tokens, weights=weights_bleu3, smoothing_function=smooth.method5)
        bleu_scores.append(score)

    # --- ROUGE-L ---
    rouge_scores = []
    for cand, ref in zip(candidates, references):
        score = rouge.score(ref, cand)
        rouge_scores.append(score['rougeL'].fmeasure)

    # --- METEOR ---
    meteor_scores = []
    for ref, cand in zip(references, candidates):
        ref_tokens = word_tokenize(ref.lower())
        cand_tokens = word_tokenize(cand.lower())
        score = meteor_score([ref_tokens], cand_tokens)
        meteor_scores.append(score)

    # --- SBERT Cosine Similarity ---
    emb_candidates = sbert_model.encode(candidates, convert_to_tensor=True)
    emb_references = sbert_model.encode(references, convert_to_tensor=True)
    cosine_scores = util.cos_sim(emb_candidates, emb_references)
    sbert_scores = [float(cosine_scores[i][i]) for i in range(len(candidates))]

    # --- Create DataFrame ---
    df = pd.DataFrame({
        "Sno": range(1, len(candidates) + 1),
        "Generated_NL": candidates,
        "Ground_NL": references,
        "BLEU3": bleu_scores,
        "ROUGE_L": rouge_scores,
        "METEOR": meteor_scores,
        "SBERT": sbert_scores
    })

    # --- Compute and append averages ---
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_rouge = sum(rouge_scores) / len(rouge_scores)
    avg_meteor = sum(meteor_scores) / len(meteor_scores)
    avg_sbert = sum(sbert_scores) / len(sbert_scores)

    df.loc[len(df)] = [
        "Average", "", "",
        avg_bleu, avg_rouge, avg_meteor, avg_sbert
    ]

    # --- Save to CSV ---
    out_path = f"Scores_{tag}.csv"
    df.to_csv(out_path, index=False, encoding="utf-8")

    print(f"✅ Scores for {tag} saved to '{out_path}'")
    print(f"🔹 Avg BLEU3: {avg_bleu:.4f} | ROUGE-L: {avg_rouge:.4f} | METEOR: {avg_meteor:.4f} | SBERT: {avg_sbert:.4f}")